In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
#from typing import Optional, Literal
#from predict.prediction import predict
import uvicorn
from uvicorn import run
import joblib
#from preprocessing.cleaning_data import preprocess
import pandas as pd
#from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from typing import Dict, Union
from category_encoders import OneHotEncoder

In [ ]:
lr_from_joblib = joblib.load('My_model.pkl')
encod = joblib.load('Encoded.pkl')

In [ ]:
class Property(BaseModel):
    region: str
    property_type: str
    habitable_surface: float
    number_of_rooms: int
    has_terrace: str
    has_garden: str
    is_furnished: str
    has_swimming_pool: str
    has_open_fire: str

In [ ]:
app = FastAPI()

In [ ]:
# Route
@app.get('/')
def get_request():
    return 'status alive'

In [ ]:
@app.post('/prediction')
def price_prediction(data_input: Property):
    try:
        # get the input data
        data = pd.DataFrame([data_input.dict()])
        data = data.replace({'Yes': 1, 'No': 0})
        data_encoded = encod.transform(data)
       
        # my prediction - with my model
        prediction = lr_from_joblib.predict(data_encoded)[0]
        # dict for the response
        response = {'prediction': prediction}

        return response
  
    except KeyError as e:
        raise HTTPException(status_code = 500, detail= str(e))

In [ ]:
description =  {"How to use": {
                        "region": "Brussels or Flanders or Wallonie",
                        "property_type": "HOUSE or APARTEMENT",
                        "habitable_surface": "requared amount of square meter for instance - 95",
                        "number_of_rooms": "requared number of rooms for instance - 3",
                        "has_terrace": "'Yes' or 'No'", 
                        "has_garden": "'Yes' or 'No'",
                        "is_furnished": "'Yes' or 'No'",
                        "has_swimming_pool": "'Yes' or 'No'",
                        "has_open_fire": "'Yes' or 'No'",
    }}

# Route
@app.get('/description')
def get_description():
    return description

In [ ]:
if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)